In [ ]:
# Setup

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

import utils


REPO_DIR = os.path.dirname(os.path.abspath("__file__"))
STORAGE_DIR = os.path.join(REPO_DIR, 'storage')
PLOTS_DIR = os.path.join(REPO_DIR, 'plots')

In [ ]:
# Utility Functions

def load_scalars(logdir, tag):
    ea = event_accumulator.EventAccumulator(logdir)
    ea.Reload()
    events = ea.Scalars(tag)
    steps = [e.step for e in events]
    values = [e.value for e in events]
    return dict(zip(steps, values))


def cut_frames(data, cut):

    if cut is None:
        return data

    cut_data = []

    for step in data.keys():
        if step <= cut:
            cut_data.append((step, data[step]))

    steps, values = zip(*cut_data)
    return dict(zip(steps, values))


def compute_average(runs):

    if len(runs) == 1:
        return runs[0]

    common_steps = set.intersection(*[set(d.keys()) for d in runs])
    common_steps = sorted(common_steps)

    averaged = []
    for step in common_steps:
        values = [d[step] for d in runs]
        avg = np.mean(values)
        averaged.append((step, avg))

    steps, values = zip(*averaged)
    return dict(zip(steps, values))


def smooth(values, alpha=0.9):

    if alpha == None:
        return values

    smoothed = []
    last = values[0]
    for v in values:
        last = alpha * last + (1 - alpha) * v
        smoothed.append(last)
    return smoothed


def save_plot(plt, name):
    os.makedirs(PLOTS_DIR, exist_ok=True)
    plt.savefig(os.path.join(PLOTS_DIR, f'{name}.pdf'))


def infer_labels(logdirs):
    return [experiment[0].split('/')[-1] for experiment in logdirs]

In [ ]:
# Plot function

def plot(logdirs, title, tag, use_legend, labels, plot_name, ylabel, xlabel='Frames (millions)', frames_range=None, smoothing=None, dimensions=(4,3.2)):

    plt.style.use("seaborn-paper")
    plt.figure(figsize=dimensions)

    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "serif",
        "font.size": 11,
        "axes.grid": True,
        "grid.alpha": 0.3,
        "grid.linestyle": "-",
        "grid.color": "gray",
        "legend.frameon": True,
        "legend.edgecolor": "black",
    })

    if labels == []:
        labels = infer_labels(logdirs)

    datas = []
    for experiment in logdirs:

        runs = []
        for run in experiment:

            run = os.path.join(STORAGE_DIR, run)
            data = load_scalars(run, tag)
            runs.append(data)

        averaged = compute_average(runs)
        averaged = cut_frames(averaged, frames_range)
        steps, values = list(averaged.keys()), list(averaged.values())
        steps = [step / 1e6 for step in steps]
        values = smooth(values, smoothing)

        plt.plot(
            steps, values,
            label=labels[0],
            linewidth=2,
            alpha=0.9
        )


    plt.xlabel(xlabel, fontsize=11, fontweight="normal")
    plt.ylabel(ylabel, fontsize=11, fontweight="normal")
    plt.title(title, fontsize=11, fontweight="normal")

    plt.grid(True, linestyle="--", alpha=0.6)
    if use_legend:
        plt.legend(loc="best", fontsize=10, frameon=True)
    plt.tight_layout()

    save_plot(plt, plot_name)
    plt.show()

In [ ]:
# Partially-Ordered Pretrain

plot(

    logdirs = [
        ['final_gnn_pretrain_e/train', 'final_gnn_pretrain_e_2/train', 'final_gnn_pretrain_e_3/train'],
    ],

    title = 'Partially-Ordered Tasks',
    plot_name = 'pretrain_e',
    ylabel = 'Discounted Return',
    tag = 'average_discounted_return',
    use_legend = False,
    labels = [],

)

In [ ]:
# Global Avoidance Pretrain

plot(

    logdirs = [
        ['final_gnn_pretrain_ga/train', 'final_gnn_pretrain_ga_2/train', 'final_gnn_pretrain_ga_3/train'],
    ],

    plot_name = 'pretrain_ga',
    title = 'Global Avoidance Tasks',
    ylabel = 'Discounted Return',
    tag = 'average_discounted_return',
    use_legend = False,
    labels = [],

)